In [1]:
import logging
import warnings
import datalink
import sys

warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s | %(message)s',
                    level=logging.DEBUG,
                    stream=sys.stdout)
log = logging.getLogger(__name__)

In [2]:
# Datalink supports the logging module.
datalink.test_output()

# Datalink logging can be disabled with
# logging.getLogger('datalink').propagate = False

INFO | logging from datalink


## Generate user classes with the `link_factory`

A data store class is generated by the link_factory method.

The user can then create instances of this class in a similar way to 

creating `namedtuple` objects from the `collections` module.

In [3]:
MyStore = datalink.link_factory(name='MyStore', db_path='/tmp/test.db', table_name='data',
                                data_fields={'a': None, 'b': [2,4], 'c': 'a string'})

In [4]:
# Databases are created as needed.
d = MyStore()

INFO | - db created at path: /tmp/test.db
DEBUG | Creating new database entry with id 17e1284a-2635-48b3-884a-052eb32453a5.


# Accessing data fields.

The internal data store is exposed via the data property.


In [5]:
d.data

{'a': None,
 'b': [2, 4],
 'c': 'a string',
 'id': '17e1284a-2635-48b3-884a-052eb32453a5'}

### Data is accessible through properties corresponding to the field name.

In [6]:
print(d.a)
d.b
d.c

None


[2, 4]

'a string'

## Data store auto saving.

The first time a property in the store is changed from its default value,

the store is persisted in the SQL database linked to the class.

Any time a field property is directly assigned, if the value is changed 

the data is saved to the SQL db.

In [7]:
# No change in data store state, so no save call.
d.a = None
d.c = 'a string'

In [8]:
# New data store state, the data is auto saved.
d.a = 12
d.data

DEBUG | Updating existing database entry for id 17e1284a-2635-48b3-884a-052eb32453a5.


{'a': 12,
 'b': [2, 4],
 'c': 'a string',
 'id': '17e1284a-2635-48b3-884a-052eb32453a5'}

In [9]:
# The update method can be used to set multiple field values 
# with only one update pushed to the SQL db.
d.update(a=14, c='a new string')
d.data

DEBUG | Updating existing database entry for id 17e1284a-2635-48b3-884a-052eb32453a5.


{'a': 14,
 'b': [2, 4],
 'c': 'a new string',
 'id': '17e1284a-2635-48b3-884a-052eb32453a5'}

By default, the class is designed to handle unique data, accessible

via a uuid generated with `uuid.uuid4`.

In [10]:
my_uuid = d.uuid
my_uuid

AttributeError: uuid

### Data store fields can be assigned in the instance declaration.

In [ ]:
d2 = MyStore(a=20, b = [6,7], c='yet another string')
d2.data

## Loading data into a new instance of MyClass.

To load previously saved data, pass the uuid as the `datalink_uuid`

keyword arg to the instance declaration.

In [ ]:
del(d, d2)
d3 = MyStore(datalink_uuid=my_uuid)

In [ ]:
d3.data

In [ ]:
# Link to the entry in the SQL db is maintained.
d3.a = 16
d3.data